In [1235]:
import numpy
import sys
from collections import defaultdict
sys.path.append('../')

In [1261]:
from util.image import convert_to_grayscale
from util.filters import make_gaussian_kernel, filter_2d, roberts_cross
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [1257]:
kernel = make_gaussian_kernel(5, 5)
blurred_gray = filter_2d(conv2d, kernel)

In [1258]:
canny = canny_edge_detection(blurred_gray, t=0.1)
y_coords, x_coords = np.where(canny)

In [1260]:
HoughCircle = HoughCircleAccumulator(x_coords, y_coords,
                                     radius_min=18, radius_max=35)
circles = HoughCircle.createCircles(100, threshold=0.25)
circles

[(19, 25, 21),
 (20, 21, 20),
 (21, 21, 20),
 (21, 29, 20),
 (24, 21, 23),
 (25, 20, 23),
 (32, 32, 24)]

In [ ]:
rho_min = -edges.shape[0]
rho_max = edges.shape[1]
hough = HoughAccumulator(128, 128, rho_min, rho_max)
accumulator = hough.accumulate(x_coords, y_coords)
max_value = np.max(accumulator)

In [ ]:
#Indices of maximum theta and rho values
rho_max_indices, theta_max_indices = np.where(accumulator > 0.6 * max_value)
thetas_max = hough.thetas[theta_max_indices]
rhos_max = hough.rhos[rho_max_indices]
l_thetas = len(thetas_max)
slope = calculate_slope_differentials(thetas_max)
thetas_max, rhos_max

In [414]:
import collections
class HoughCircleAccumulator():
    def __init__(self,x_coords, y_coords, radius_min=70, radius_max=75):
            self.x_coords = x_coords
            self.y_coords = y_coords
            self.radius_min = radius_min
            self.radius_max = radius_max
            self.accumulate = collections.defaultdict(int)
            
    def createCircles(self, n_iter, threshold=0.3):
        points = []
        # Create Possible Points
        for r in range(self.radius_min, self.radius_max):
            for t in range(n_iter):
                points.append((r, int(r * cos(2 * pi * t / n_iter)), int(r * sin(2 * pi * t / n_iter))))

        # Hough Points on Space
        for x, y in zip(self.x_coords, self.y_coords):
            for r, dx, dy in points:
                a = x - dx
                b = y - dy
                # possible centroids
                self.accumulate[(a, b, r)] += 1
        
        circles = self.threshold_accumulator(n_iter, threshold=threshold)
        return circles
        
    def threshold_accumulator(self, n_iter, threshold=0.3):
        circles = []
        for key, val in sorted(self.accumulate.items()):
            x, y, r = key
            if (val / n_iter) > threshold and all((x - xc) ** 2 + (y - yc) ** 2 > rc ** 2 for xc, yc, rc in circles):
                circles.append((x, y, r))
        return circles
    
    def calculate_max_occurences(circles):
        count = collections.defaultdict(int)
        maxCircleOcc = 0
        center = (0,0)
        for x, y, r in circles:
            count[(x,y)] += 1
            if count[(x,y)] > maxCircleOcc:
                maxCircleOcc = count[(x,y)]
                center = (x,y)

        rangeLo = float('inf')
        rangeHi = float('-inf')
        for k, v in count.items():
            a, b = k
            rangeLo = min(rangeLo, min(a, b))
            rangeHi = max(rangeHi, max(a, b))
        return (rangeLo, rangeHi, center)

def calculate_x_y(blurred_image, threshold=0.03):
    # Sobel Feldman
    Kx = np.array([[1, 0, -1], 
          [2, 0, -2],
          [1, 0, -1]])
    Ky = np.array([[1, 2, 1],
          [0, 0, 0],
          [-1, -2, -1]])
    Gx = filter_2d(blurred_gray, Kx)
    Gy = filter_2d(blurred_gray, Ky)

    G = np.sqrt(Gx**2+ Gy**2)
    thresh = threshold
    edges = G > thresh
    y_coords, x_coords = np.where(edges)
    return edges, y_coords, x_coords

def max_pool(input_im):
    h, w = input_im.shape
    new_h = h//2
    new_w = w//2
    reduced = np.zeros((new_h, new_w))
    for i in range(new_h):
      for j in range(new_w):
        reduced[i][j] = np.max(input_im[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)])
    
    return reduced

def canny_edge_detection(convolved_image, t):
    G, G_direction = sobel_feldmans(convolved_image)
    sup = non_max_suppression(G, G_direction)
    # left out hysteria's -- too bored to finish
    ret = sup > t
    return ret

def non_max_suppression(gradient_magnitude, gradient_direction):
    image_row, image_col = gradient_magnitude.shape
    output = np.zeros(gradient_magnitude.shape)
    PI = 180	
    
    # check all sides bi-directional to current magnitude
    for row in range(1, image_row - 1):
        for col in range(1, image_col - 1):
            direction = gradient_direction[row, col]

            if (0 <= direction < PI / 8) or (15 * PI / 8 <= direction <= 2 * PI):
                before_pixel = gradient_magnitude[row, col - 1]
                after_pixel = gradient_magnitude[row, col + 1]

            elif (PI / 8 <= direction < 3 * PI / 8) or (9 * PI / 8 <= direction < 11 * PI / 8):
                before_pixel = gradient_magnitude[row + 1, col - 1]
                after_pixel = gradient_magnitude[row - 1, col + 1]

            elif (3 * PI / 8 <= direction < 5 * PI / 8) or (11 * PI / 8 <= direction < 13 * PI / 8):
                before_pixel = gradient_magnitude[row - 1, col]
                after_pixel = gradient_magnitude[row + 1, col]

            else:
                before_pixel = gradient_magnitude[row - 1, col - 1]
                after_pixel = gradient_magnitude[row + 1, col + 1]

            if gradient_magnitude[row, col] >= before_pixel and gradient_magnitude[row, col] >= after_pixel:
                output[row, col] = gradient_magnitude[row, col]
    return output

def sobel_feldmans(blurred_image):
    # Sobel-Feldman Filters
    Kx = np.array([[5, 0, -5], 
          [7, 0, -7],
          [5, 0, -5]])
    Ky = np.array([[5, 7, 5],
          [0, 0, 0],
          [-5, -7, -5]])
    Gx = filter_2d(blurred_image, Kx)
    Gy = filter_2d(blurred_image, Ky)
    
    G_direction = np.arctan2(Gx, Gy)
    G = np.sqrt(Gx**2+ Gy**2)
    return G, G_direction